In [5]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import locale
from locale import atof

In [6]:
# Load in csv
filepath = os.path.join("CSV Files\\TWFC CSV\\Employment.csv")
employment_numbers_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\TWFC CSV\\Un_numbers.csv")
unemployment_numbers_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\TWFC CSV\\Un_rate.csv")
unemployment_rate_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\TWFC CSV\\Laborforce.csv")
laborforce_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\TWFC CSV\\Income.csv")
income_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\TWFC CSV\\Population.csv")
population_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\Zillow CSV\\Price.csv")
price_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\Zillow CSV\\Rent.csv")
rent_df = pd.read_csv(filepath)

In [7]:
# Merge the monthly CSVs
merge_df = pd.merge(employment_numbers_df, unemployment_numbers_df, how="outer", on=["Year", "Period ID", "Period", "Area"])
merge_df2 = pd.merge(merge_df, unemployment_rate_df, how="outer", on=["Year", "Period ID", "Period", "Area"])
merge_df3 = pd.merge(merge_df2, laborforce_df, how="outer", on=["Year", "Period ID", "Period", "Area"])

# Select desired columns 
merge_df = merge_df3.drop(merge_df3.columns[[2,4,6,8,10]], axis=1)

# Change column name of Area to City
merge_df=merge_df.rename(columns = {'Area':'City', 'Period ID': 'Month'})

# Slice out MSA out of the City Names
merge_df["City"] = merge_df["City"].replace(' MSA','',regex=True)
merge_df = merge_df

# Convert Strings to Floats
merge_df["Employment"] = merge_df["Employment"].replace(',','',regex=True).astype('float')
merge_df["Unemployment"] = merge_df["Unemployment"].replace(',','',regex=True).astype('float')
merge_df["Labor Force"] = merge_df["Labor Force"].replace(',','',regex=True).astype('float')

#Clean up city names
merge_df['City'], merge_df['x'] = merge_df["City"].str.split('-', 2).str[0:2].str
merge_df = merge_df.drop(merge_df.columns[[-1]], axis=1)

# Convert Region Name into Seperate City and State Columns
rent_df['City'],rent_df['State'] = rent_df["RegionName"].str.split(',', 2).str[0:2].str
rent_df['State'].replace(' TX','TX', inplace=True)
price_df['City'],price_df['State'] = price_df["RegionName"].str.split(',', 2).str[0:2].str
price_df['State'].replace(' TX','TX', inplace=True)

# Extract only the Texas Data
rent_tx_df = rent_df.loc[rent_df["State"] == "TX", :]
rent_tx_df = rent_tx_df.set_index('City')
price_tx_df = price_df.loc[price_df["State"] == "TX", :]
price_tx_df = price_tx_df.set_index('City')

# Remove Undesired Columns
rent_tx_df = rent_tx_df.drop(rent_tx_df.columns[[0,1,2,-1]], axis=1)
price_tx_df = price_tx_df.drop(price_tx_df.columns[[0,1,-1]], axis=1)

# Transpose Data Frame
rent = rent_tx_df.transpose()
price = price_tx_df.transpose()

#Move the Index
rent.reset_index(level=0, inplace=True)
rent = rent.rename(columns = {'index':'Date'})
rent = rent.reset_index(drop=True)
price.reset_index(level=0, inplace=True)
price = price.rename(columns = {'index':'Date'})
price = price.reset_index(drop=True)

# Convert the Date into a Month and Year Column
rent['Year'],rent['Month'] = rent["Date"].str.split('-', 2).str[0:2].str
rent = rent.drop(rent.columns[[0]], axis=1)
price['Year'],price['Month'] = price["Date"].str.split('-', 2).str[0:2].str
price = price.drop(price.columns[[0]], axis=1)

# Melt Data Frames and Restructure and Merge
rent_melt = pd.melt(rent, id_vars=["Year", "Month"])
rent_melt = rent_melt.rename(columns={'value': 'Rent'})
rent_melt = rent_melt.replace('Dallas-Fort Worth', 'Dallas')
price_melt = pd.melt(price, id_vars=["Year", "Month"])
price_melt = price_melt.rename(columns={'value': 'House Price'})
merge_melts = pd.merge(rent_melt,price_melt, how='left', on=['Year','Month','City'])

# Merge TWFC Data with Zillow Data
monthly_merge = pd.merge(merge_df, merge_melts, how="outer", on=["Year", "Month", "City"])

# Clean up Income data to merge to annual data frame
# Select desired columns 
income_df = income_df.drop(income_df.columns[[1,3,4]], axis=1)

# Change column name of Area to City
income_df = income_df.rename(columns = {'Area':'City'})

# Slice out MSA out of the City Names
income_df["City"] = income_df["City"].replace(' MSA','',regex=True)

# Convert Strings to Floats
income_df["Income"] = income_df["Income"].replace(',','',regex=True)

income = []
for x in income_df["Income"]:
    x = x[1:]
    x = int(x)
    income.append(x)

income_df["Income"] = income

# Merge Income Data Frame to the Main Data Frame
df_final = pd.merge(monthly_merge, income_df, how="outer", on=["City", "Year"])
df_final = df_final.groupby(['City', 'Year']).mean()
df_final = df_final.drop(df_final.columns[[0]], axis=1)
df_final

Employment  Unemployment  Unemployment Rate  \
City          Year                                                     
Abilene       1990.0   64531.416667   4052.750000           5.916667   
              1991.0   65709.333333   3668.083333           5.283333   
              1992.0   67167.250000   3907.083333           5.491667   
              1993.0   68869.916667   3901.083333           5.350000   
              1994.0   70369.500000   3630.000000           4.908333   
              1995.0   72515.833333   3680.333333           4.816667   
              1996.0   73482.083333   3550.000000           4.608333   
              1997.0   73689.416667   2946.416667           3.841667   
              1998.0   74039.750000   2788.250000           3.625000   
              1999.0   72962.000000   2856.750000           3.766667   
              2000.0   72335.250000   3098.333333           4.100000   
              2001.0   72004.750000   3348.750000           4.450000   
              2002.0   73769.750000   3944.583333           5.066667   
              2003.0   75352.166667   4223.416667           5.316667   
              2004.0   75873.250000   3703.916667           4.658333   
              2005.0   76723.500000   3376.500000           4.216667   
              2006.0   78166.416667   3242.583333           3.991667   
              2007.0   77188.500000   2825.500000           3.525000   
              2008.0   78881.333333   3106.833333           3.791667   
              2009.0   77873.583333   4806.333333           5.808333   
              2010.0   71121.750000   5326.750000           6.958333   
              2011.0   71638.000000   5184.083333           6.750000   
              2012.0   71963.416667   4441.666667           5.808333   
              2013.0   72512.333333   4110.416667           5.375000   
              2014.0   72768.750000   3293.750000           4.341667   
              2015.0   72228.833333   2915.833333           3.900000   
              2016.0   72864.333333   2923.750000           3.866667   
              2017.0   71511.777778   2976.333333           3.988889   
Alice         2010.0            NaN           NaN                NaN   
              2011.0            NaN           NaN                NaN   
...                             ...           ...                ...   
Waco          2016.0  116284.083333   4829.333333           3.983333   
              2017.0  117539.444444   5130.777778           4.166667   
Wichita Falls 1990.0   61418.000000   4135.333333           6.308333   
              1991.0   60580.666667   4348.916667           6.708333   
              1992.0   61060.666667   4148.416667           6.358333   
              1993.0   62913.500000   3684.750000           5.525000   
              1994.0   64472.333333   3488.916667           5.133333   
              1995.0   66719.166667   3224.250000           4.625000   
              1996.0   67804.416667   3096.166667           4.383333   
              1997.0   68044.083333   3063.416667           4.316667   
              1998.0   67611.000000   3329.000000           4.700000   
              1999.0   66608.333333   3179.000000           4.550000   
              2000.0   68669.000000   3157.666667           4.408333   
              2001.0   70009.000000   3148.166667           4.308333   
              2002.0   70096.833333   4204.166667           5.658333   
              2003.0   70483.666667   4289.166667           5.750000   
              2004.0   70694.083333   3662.083333           4.900000   
              2005.0   70899.750000   3374.083333           4.541667   
              2006.0   71407.416667   3150.166667           4.225000   
              2007.0   70114.333333   2947.083333           4.016667   
              2008.0   68876.833333   3380.666667           4.675000   
              2009.0   67139.166667   5329.500000           7.350000   
              2010.0   64099.333333   5333.250000           7.675